In [88]:
# Donwload libraries
!pip install scikit-learn
!pip install seaborn
!pip install numpy
!pip install pandas
!pip install scikit-multilearn
!pip install kagglehub
!git lfs install
!git lfs pull
!pip install plotly

Updated Git hooks.
Git LFS initialized.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.1/19.1 MB 4.6 MB/s eta 0:00:0000:0100:01


# Importing libraries and packages

In [99]:
#  git lfs track "*.csv" to track all csv files

import os
import glob
import kagglehub
import sqlite3
#Data Wrangling

import numpy as np
import pandas as pd

#Trees
from sklearn import tree
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

#Preprocessing Packages
from sklearn.model_selection import train_test_split

#Cross Validation
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

#Model tuning
from sklearn.model_selection import GridSearchCV

#Metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import confusion_matrix

#Plotting
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

%matplotlib inline

Loading Data Sets

In [90]:
# set up respective path to the data
project_root = os.path.abspath(os.path.join(".."))
print(project_root)
data_path = os.path.join(project_root, "raw-data/")
print(data_path)

# concatenate all seasons of data in a single df
df = pd.DataFrame()
files = glob.glob(os.path.join(data_path, '*_Shots.csv'))
# for file in os.listdir(data_path):
#     file_path=os.path.join(data_path,file)
#     data=pd.read_csv(file_path)
#     df=pd.concat([df,data],ignore_index=True)
#     print(file)
df = pd.concat((pd.read_csv(file) for file in files), ignore_index=True)
df.head()

/Users/fant1c/DataspellProjects/project-cis-5450
/Users/fant1c/DataspellProjects/project-cis-5450/raw-data/


,SEASON_1,SEASON_2,TEAM_ID,TEAM_NAME,PLAYER_ID,PLAYER_NAME,POSITION_GROUP,POSITION,GAME_DATE,GAME_ID,...,BASIC_ZONE,ZONE_NAME,ZONE_ABB,ZONE_RANGE,LOC_X,LOC_Y,SHOT_DISTANCE,QUARTER,MINS_LEFT,SECS_LEFT
0,2009,2008-09,1610612744,Golden State Warriors,201627,Anthony Morrow,G,SG,04-15-2009,20801229,...,Restricted Area,Center,C,Less Than 8 ft.,-0.0,5.25,0,4,0,1
1,2009,2008-09,1610612744,Golden State Warriors,101235,Kelenna Azubuike,F,SF,04-15-2009,20801229,...,Restricted Area,Center,C,Less Than 8 ft.,-0.0,5.25,0,4,0,9
2,2009,2008-09,1610612756,Phoenix Suns,255,Grant Hill,F,SF,04-15-2009,20801229,...,Restricted Area,Center,C,Less Than 8 ft.,-0.0,5.25,0,4,0,25
3,2009,2008-09,1610612739,Cleveland Cavaliers,200789,Daniel Gibson,G,PG,04-15-2009,20801219,...,Restricted Area,Center,C,Less Than 8 ft.,-0.2,5.25,0,5,0,4
4,2009,2008-09,1610612756,Phoenix Suns,255,Grant Hill,F,SF,04-15-2009,20801229,...,Mid-Range,Left Side,L,8-16 ft.,8.7,7.55,8,4,1,3


# Data Dictionary

**Self-Explanatory**: **TEAM_NAME**, **PLAYER_NAME**, **POSITION_GROUP**, **POSITION**, **HOME_TEAM**, **AWAY_TEAM**

- **SEASON_1** & **SEASON_2**: Season indicator variables  
- **TEAM_ID**: NBA's unique ID variable of that specific team in their API  
- **PLAYER_ID**: NBA's unique ID variable of that specific player in their API  
- **GAME_DATE**: Date of the game (M-D-Y // Month-Date-Year)  
- **GAME_ID**: NBA's unique ID variable of that specific game in their API  
- **EVENT_TYPE**: Character variable denoting a shot outcome (Made Shot // Missed Shot)  
- **SHOT_MADE**: True/False variable denoting a shot outcome (True // False)  
- **ACTION_TYPE**: Description of shot type (layup, dunk, jump shot, etc.)  
- **SHOT_TYPE**: Type of shot (2PT or 3PT)  
- **BASIC_ZONE**: Name of the court zone the shot took place in  
  - Restricted Area, In the Paint (non-RA), Midrange, Left Corner 3, Right Corner 3, Above the Break, Backcourt  
- **ZONE_NAME**: Name of the side of court the shot took place in  
  - left, left side center, center, right side center, right  
- **ZONE_ABB**: Abbreviation of the side of court  
  - (L), (LC), (C), (RC), (R)  
- **ZONE_RANGE**: Distance range of shot by zones  
  - Less than 8 ft., 8-16 ft. 16-24 ft. 24+ ft.  
- **LOC_X**: X coordinate of the shot in the x, y plane of the court (0, 50)  
- **LOC_Y**: Y coordinate of the shot in the x, y plane of the court (0, 50)  
- **SHOT_DISTANCE**: Distance of the shot with respect to the center of the hoop, in feet  
- **QUARTER**: Quarter of the game  
- **MINS_LEFT**: Minutes remaining in the quarter  
- **SECS_LEFT**: Seconds remaining in minute of the quarter  


In [91]:
# inspect the Leagues data set
print("Number of duplicate rows: " + str(df.duplicated().sum()))
print("dtypes:")
print(df.dtypes)
print("----------------------------\nLeagues DF describe:")
print(df.describe())
print("----------------------------\nCheck for Null values:")
print(df.isna().sum())
print("----------------------------\nFirst 5 row of the dataset")
df.head()

Number of duplicate rows: 192
dtypes:
SEASON_1            int64
SEASON_2           object
TEAM_ID             int64
TEAM_NAME          object
PLAYER_ID           int64
PLAYER_NAME        object
POSITION_GROUP     object
POSITION           object
GAME_DATE          object
GAME_ID             int64
HOME_TEAM          object
AWAY_TEAM          object
EVENT_TYPE         object
SHOT_MADE            bool
ACTION_TYPE        object
SHOT_TYPE          object
BASIC_ZONE         object
ZONE_NAME          object
ZONE_ABB           object
ZONE_RANGE         object
LOC_X             float64
LOC_Y             float64
SHOT_DISTANCE       int64
QUARTER             int64
MINS_LEFT           int64
SECS_LEFT           int64
dtype: object
----------------------------
Leagues DF describe:
           SEASON_1       TEAM_ID     PLAYER_ID       GAME_ID         LOC_X  \
count  4.231262e+06  4.231262e+06  4.231262e+06  4.231262e+06  4.231262e+06   
mean   2.014191e+03  1.610613e+09  4.136611e+05  2.131969e+07  9

,SEASON_1,SEASON_2,TEAM_ID,TEAM_NAME,PLAYER_ID,PLAYER_NAME,POSITION_GROUP,POSITION,GAME_DATE,GAME_ID,...,BASIC_ZONE,ZONE_NAME,ZONE_ABB,ZONE_RANGE,LOC_X,LOC_Y,SHOT_DISTANCE,QUARTER,MINS_LEFT,SECS_LEFT
0,2009,2008-09,1610612744,Golden State Warriors,201627,Anthony Morrow,G,SG,04-15-2009,20801229,...,Restricted Area,Center,C,Less Than 8 ft.,-0.0,5.25,0,4,0,1
1,2009,2008-09,1610612744,Golden State Warriors,101235,Kelenna Azubuike,F,SF,04-15-2009,20801229,...,Restricted Area,Center,C,Less Than 8 ft.,-0.0,5.25,0,4,0,9
2,2009,2008-09,1610612756,Phoenix Suns,255,Grant Hill,F,SF,04-15-2009,20801229,...,Restricted Area,Center,C,Less Than 8 ft.,-0.0,5.25,0,4,0,25
3,2009,2008-09,1610612739,Cleveland Cavaliers,200789,Daniel Gibson,G,PG,04-15-2009,20801219,...,Restricted Area,Center,C,Less Than 8 ft.,-0.2,5.25,0,5,0,4
4,2009,2008-09,1610612756,Phoenix Suns,255,Grant Hill,F,SF,04-15-2009,20801229,...,Mid-Range,Left Side,L,8-16 ft.,8.7,7.55,8,4,1,3


In [92]:
df_clean = df.drop_duplicates()
df_clean.head()

,SEASON_1,SEASON_2,TEAM_ID,TEAM_NAME,PLAYER_ID,PLAYER_NAME,POSITION_GROUP,POSITION,GAME_DATE,GAME_ID,...,BASIC_ZONE,ZONE_NAME,ZONE_ABB,ZONE_RANGE,LOC_X,LOC_Y,SHOT_DISTANCE,QUARTER,MINS_LEFT,SECS_LEFT
0,2009,2008-09,1610612744,Golden State Warriors,201627,Anthony Morrow,G,SG,04-15-2009,20801229,...,Restricted Area,Center,C,Less Than 8 ft.,-0.0,5.25,0,4,0,1
1,2009,2008-09,1610612744,Golden State Warriors,101235,Kelenna Azubuike,F,SF,04-15-2009,20801229,...,Restricted Area,Center,C,Less Than 8 ft.,-0.0,5.25,0,4,0,9
2,2009,2008-09,1610612756,Phoenix Suns,255,Grant Hill,F,SF,04-15-2009,20801229,...,Restricted Area,Center,C,Less Than 8 ft.,-0.0,5.25,0,4,0,25
3,2009,2008-09,1610612739,Cleveland Cavaliers,200789,Daniel Gibson,G,PG,04-15-2009,20801219,...,Restricted Area,Center,C,Less Than 8 ft.,-0.2,5.25,0,5,0,4
4,2009,2008-09,1610612756,Phoenix Suns,255,Grant Hill,F,SF,04-15-2009,20801229,...,Mid-Range,Left Side,L,8-16 ft.,8.7,7.55,8,4,1,3


In [93]:
#TODO:
print("Number of rows containing NA values:\n" + str(df_clean.isna().sum()[df_clean.isna().sum() > 0]))
df_clean[df_clean["POSITION_GROUP"].isna() | df_clean["POSITION"].isna()]

Number of rows containing NA values:
POSITION_GROUP    7930
POSITION          7930
dtype: int64


,SEASON_1,SEASON_2,TEAM_ID,TEAM_NAME,PLAYER_ID,PLAYER_NAME,POSITION_GROUP,POSITION,GAME_DATE,GAME_ID,...,BASIC_ZONE,ZONE_NAME,ZONE_ABB,ZONE_RANGE,LOC_X,LOC_Y,SHOT_DISTANCE,QUARTER,MINS_LEFT,SECS_LEFT
812184,2023,2022-23,1610612761,Toronto Raptors,1628384,O.G. Anunoby,NaN,NaN,10-19-2022,22200008,...,Above the Break 3,Left Side Center,LC,24+ ft.,15.0,26.95,26,1,11,23
812187,2023,2022-23,1610612761,Toronto Raptors,1628384,O.G. Anunoby,NaN,NaN,10-19-2022,22200008,...,In The Paint (Non-RA),Left Side,L,8-16 ft.,5.6,14.45,10,1,9,42
812199,2023,2022-23,1610612761,Toronto Raptors,1628384,O.G. Anunoby,NaN,NaN,10-19-2022,22200008,...,Above the Break 3,Right Side Center,RC,24+ ft.,-8.9,29.35,25,1,1,15
812207,2023,2022-23,1610612761,Toronto Raptors,1628384,O.G. Anunoby,NaN,NaN,10-19-2022,22200008,...,In The Paint (Non-RA),Center,C,Less Than 8 ft.,2.1,10.05,5,2,9,24
812222,2023,2022-23,1610612761,Toronto Raptors,1628384,O.G. Anunoby,NaN,NaN,10-19-2022,22200008,...,Restricted Area,Center,C,Less Than 8 ft.,-0.1,4.65,0,2,1,54
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4157912,2015,2014-15,1610612766,Charlotte Hornets,203106,Jeffery Taylor,NaN,NaN,12-26-2014,21400438,...,Restricted Area,Center,C,Less Than 8 ft.,-0.1,7.45,2,4,2,49
4157940,2015,2014-15,1610612766,Charlotte Hornets,203106,Jeffery Taylor,NaN,NaN,12-26-2014,21400438,...,Mid-Range,Right Side,R,8-16 ft.,-11.9,5.05,11,4,4,5
4158003,2015,2014-15,1610612766,Charlotte Hornets,203106,Jeffery Taylor,NaN,NaN,12-26-2014,21400438,...,Restricted Area,Center,C,Less Than 8 ft.,-1.5,5.05,1,4,6,59
4162561,2015,2014-15,1610612766,Charlotte Hornets,203106,Jeffery Taylor,NaN,NaN,12-22-2014,21400409,...,Right Corner 3,Right Side,R,24+ ft.,-23.6,6.35,23,4,2,14


In [94]:
conn = sqlite3.connect("database.db")
df_clean.to_sql("nba", conn, if_exists="replace", index=False)


4231070

# Exploratory DAta Analysis (EDA)
- Jump Shot accuracy by Zone (2pt and 3pt)
- Accuracy by action type
- Most efficient players + shot distribution heat map
- Home vs. Performance for teams
- Time trend in accuracy, points,  action type, etc

# Accuracy by action type

In [95]:
action_types_p = df_clean.ACTION_TYPE.unique()
action_types_sql = pd.read_sql("SELECT DISTINCT ACTION_TYPE from nba", conn)
print("Pandas and SQL versions are identical: " + str(len(action_types_p) == len(action_types_sql)))
action_types_p

Pandas and SQL versions are identical: True


array(['Driving Layup Shot', 'Layup Shot', 'Jump Shot',
       'Running Bank shot', 'Reverse Layup Shot', 'Jump Bank Shot',
       'Tip Shot', 'Slam Dunk Shot', 'Hook Shot', 'Running Jump Shot',
       'Dunk Shot', 'Step Back Jump shot', 'Putback Layup Shot',
       'Floating Jump shot', 'Driving Reverse Layup Shot',
       'Driving Dunk Shot', 'Reverse Dunk Shot', 'Pullup Jump shot',
       'Turnaround Jump Shot', 'Alley Oop Dunk Shot',
       'Running Layup Shot', 'Jump Hook Shot', 'Driving Bank shot',
       'Alley Oop Layup shot', 'Putback Dunk Shot', 'Fadeaway Jump Shot',
       'Turnaround Fadeaway shot', 'Finger Roll Layup Shot',
       'Driving Slam Dunk Shot', 'Pullup Bank shot',
       'Turnaround Hook Shot', 'Driving Jump shot',
       'Turnaround Bank shot', 'Running Hook Shot',
       'Driving Finger Roll Layup Shot', 'Fadeaway Bank shot',
       'Reverse Slam Dunk Shot', 'Running Dunk Shot',
       'Running Reverse Layup Shot', 'Putback Slam Dunk Shot',
       'Driving Ho

In [113]:
# query = """
# SELECT *
# FROM nba
# WHERE
#     SHOT_COUNT
# """
# action_types_sql = pd.read_sql(
# )
popular_action_types_df_p = df_clean.groupby("ACTION_TYPE").agg(count=("ACTION_TYPE", "count")).sort_values(by="count",
                                                                                                            ascending=False)
popular_action_types_df_p = popular_action_types_df_p[
    popular_action_types_df_p["count"] >= popular_action_types_df_p["count"].quantile(0.75)].reset_index()
action_types_75_p = df_clean[df_clean["ACTION_TYPE"].isin(popular_action_types_df_p["ACTION_TYPE"])][
    ["ACTION_TYPE", "SHOT_MADE", "SHOT_DISTANCE"]]
action_types_75_p = action_types_75_p.groupby("ACTION_TYPE").agg(accuracy=("SHOT_MADE", "mean"),
                                                                 avg_distance=("SHOT_DISTANCE", "mean"),
                                                                 count=("SHOT_MADE", "count")).reset_index()
action_types_75_p

,ACTION_TYPE,accuracy,avg_distance,count
0,Cutting Layup Shot,0.717048,1.701533,42926
1,Driving Finger Roll Layup Shot,0.675188,1.882761,45454
2,Driving Floating Jump Shot,0.432528,7.890536,65894
3,Driving Layup Shot,0.597654,1.508562,288734
4,Dunk Shot,0.870218,0.381341,67837
5,Fadeaway Jump Shot,0.472016,12.970257,64250
6,Floating Jump shot,0.485296,8.303880,58148
7,Hook Shot,0.449582,5.591407,66335
8,Jump Shot,0.345617,20.006034,2002674
9,Layup Shot,0.464173,0.852218,437781


In [154]:
# Bubble Chart (Scatter Plot with Bubble Size)
# using tuple unpacking for multiple Axes


fig = go.Figure()
fig.update_layout(width=1400, height=800, title="NBA Players Action Type Accuracy, Shot Distance and Attempt Count",
                  xaxis=dict(
                      title="Average Distance",
                      gridcolor="white",
                      gridwidth=2,
                  ),
                  yaxis=dict(
                      title="Accuracy (%)",
                      gridcolor="white",
                      gridwidth=2,
                  ),
                  )
fig.add_trace(
    go.Scatter(
        x=action_types_75_p["avg_distance"],
        y=action_types_75_p["accuracy"] * 100,
        mode="markers",
        marker=dict(
            size=action_types_75_p["count"],
            sizemode="area",
            sizeref=100,
            color=action_types_75_p["count"],
            colorscale="Viridis",
            showscale=True,
            line_color='rgb(140, 140, 170)',
            colorbar_title='Shots<br>Count',
        ),
    text=action_types_75_p["ACTION_TYPE"],
    hovertemplate=(
            "Action Type: %{text}<br>" +
            "Accuracy: %{y:.2f}<br>" +
            "Average Distance: %{x:.2f}<br>" +
            "Attempt Count: %{marker.size:.0f}"
    ),
    name="Action Types"
    )
)
# Stacked or Grouped Bar Chart
# Heatmap with Annotations
# 3D Scatter Plot

# Predictive Models
- Shot success and accuracy prediction model
- Players synergy and selection model ? 

In [40]:
conn.close()